---
## Admin

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (16.0, 6.0)

---
## Step 0: Load The Data

In [3]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

inputs_loc = '/media/fqaiser94/ff3daf1f-fc36-43a4-a218-478126941f2a/traffic-signs-data/'

training_file = inputs_loc + 'train.p'
validation_file= inputs_loc + 'valid.p'
testing_file = inputs_loc + 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Prep Data

In [4]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed. 

def normalize(data):
    
    data_normalized = (data - 128) / 128
    
    return data_normalized

In [5]:
X_train_norm = normalize(X_train)
X_valid_norm = normalize(X_valid)
X_test_norm = normalize(X_test)

In [6]:
print('old mean: %s' %np.mean(X_train))
print('old std:  %s' %np.std(X_train))

print('new mean: %s' %np.mean(X_train_norm))
print('new std:  %s' %np.std(X_train_norm))

old mean: 82.677589037
old std:  67.8508884263
new mean: 1.21925099939
new std:  0.449703953545


----

## Step 2: Design and Test a Model Architecture

### Model Architecture

define helper function

In [7]:
def calc_filter_length(old_length, new_length, padding, stride): 

    # OG formula
    # new_width = (old_width - filter_width + 2*padding)/stride + 1
    # rewrite to solve for filter_width
    filter_length = -1*(stride*(new_length-1) - 2*padding - old_length)    
    
    return filter_length

In [8]:
# test function 

filter_length = calc_filter_length(old_length = 32, 
                                  new_length = 28, 
                                  padding = 0, 
                                  stride = 1
                                 )
assert filter_length==5, "Houston we've got a problem"

define training pipeline/architecture. 

In [9]:
# from tensorflow.contrib.layers import flatten


# def LeNet(x):
    
#     x = tf.cast(x, tf.float32)
    
#     mu = 0
#     sigma = 0.1
    
#     # layer 1
#     # convolution
    
#     conv_w1 = tf.Variable(
#         tf.truncated_normal(
#             shape=(5,5,3,6), # width, height, input depth, output depth
#             mean=mu,
#             stddev=sigma,
#             dtype=tf.float32,
#             seed=None,
#             name=None
#         )
#     )
    
#     conv_b1 = tf.Variable(
#         tf.zeros(6)
#     )
    
#     conv1 = tf.nn.conv2d(
#         input=x, 
#         filter=conv_w1, 
#         strides=[1,1,1,1], 
#         padding='VALID'
#     ) + conv_b1
    
#     # Activation
    
#     conv1 = tf.nn.relu(conv1)
    
#     # pooling
    
#     conv1 = tf.nn.max_pool(
#         value=conv1, 
#         ksize=[1,2,2,1], 
#         strides=[1,2,2,1], 
#         padding='VALID'
#     )
    
#     # layer 2
#     # convolution

#     conv_w2 = tf.Variable(
#         tf.truncated_normal(
#             shape=(5,5,6,16), # width, height, input depth, output depth
#             mean=mu,
#             stddev=sigma,
#             dtype=tf.float32,
#             seed=None,
#             name=None
#         )
#     )
    
#     conv_b2 = tf.Variable(
#         tf.zeros(16)
#     )
    
#     conv2 = tf.nn.conv2d(
#         input=conv1, 
#         filter=conv_w2, 
#         strides=[1,1,1,1], 
#         padding='VALID'
#     ) + conv_b2
    
#     # Activation
    
#     conv2 = tf.nn.relu(
#         conv2
#     )
    
#     # pooling
    
#     conv2 = tf.nn.max_pool(
#         value=conv2, 
#         ksize=[1,2,2,1], 
#         strides=[1,2,2,1], 
#         padding='VALID'
#     )
    
#     # flatten 0
    
#     fc0 = flatten(
#         conv2
#     )
    
#     # Layer 3: Fully Connected. 
#     # Input = 400
#     # Output = 120
#     # (input, output)
#     fc1_W = tf.Variable(
#         tf.truncated_normal(
#             shape=(400, 120), 
#             mean = mu, 
#             stddev = sigma
#         )
#     )
    
#     # (output)
#     fc1_b = tf.Variable(
#         tf.zeros(120)
#     )
    
#     # regression
#     fc1 = tf.matmul(fc0, fc1_W) + fc1_b
    
#     # Activation
#     fc1 = tf.nn.relu(fc1)
    
#     # Layer 4: Fully Connected. 
#     # Input = 120
#     # Output = 84
#     # (input, output)
#     fc2_W  = tf.Variable(
#         tf.truncated_normal(
#             shape=(120, 84), 
#             mean = mu, 
#             stddev = sigma)
#     )
    
#     fc2_b  = tf.Variable(tf.zeros(84))
    
#     # regression
#     fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
#     # Activation
#     fc2    = tf.nn.relu(fc2)
    
#     # Layer 5: Fully Connected
#     # Input = 84
#     # Output = 10
#     # (input, output)
#     fc3_W  = tf.Variable(
#         tf.truncated_normal(
#             shape=(84, 43), 
#             mean = mu, 
#             stddev = sigma
#         )
#     )
    
#     # (output)
#     fc3_b  = tf.Variable(
#         tf.zeros(43)
#     )
   
#     # regression
#     logits = tf.matmul(fc2, fc3_W) + fc3_b
    
#     return logits

In [16]:
# official solution

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [17]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

In [18]:
from sklearn.utils import shuffle

# X_train, y_train = shuffle(X_train, y_train)

In [19]:
import tensorflow as tf

EPOCHS = 10
BATCH_SIZE = 128

In [20]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

In [21]:
rate = 0.001

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [22]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [23]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_valid_norm, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.061

EPOCH 2 ...
Validation Accuracy = 0.079

EPOCH 3 ...
Validation Accuracy = 0.055

EPOCH 4 ...
Validation Accuracy = 0.055

EPOCH 5 ...
Validation Accuracy = 0.052

EPOCH 6 ...
Validation Accuracy = 0.049

EPOCH 7 ...
Validation Accuracy = 0.088

EPOCH 8 ...
Validation Accuracy = 0.081

EPOCH 9 ...
Validation Accuracy = 0.089

EPOCH 10 ...
Validation Accuracy = 0.074

Model saved


In [ ]:
# Here's a recipe I copied from one of Alex Mordvintsev deep dream notebook at some point

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())
